# Speckle Automate notebook example

This is an example jupyter notebook, that will be executed by speckle automate


# Parameters

These values are supplied to the notebook, when its being run by automate.

WANING: You really shouldn't modify this block, unless its following upstream changes from automate.


In [60]:
from speckle_automate import AutomationContext, AutomationRunData

function_inputs = '{"forbiddenSpeckleType": "Objects.Geometry.Brep"}'
token_env_var = "SPECKLE_TOKEN"
automation_run_data = ''

execute_function = False

# Local testing

The cell below is here to give an easy way into locally testing the notebook.

The cell has an option to register a new automation for the project on every run otherwise
the empty values have to be replaced with valid data.

With the local testing flag enabled, the cell will override parameters from the cell above.


In [69]:
local_testing = False
register_automation = True

if local_testing:
    execute_function = True
    speckle_token = ""
    from speckle_automate.helpers import register_new_automation, crypto_random_string

    if not register_automation:
        automation_id = ""
        automation_revision_id = ""
        automation_run_id = ""
        function_id = ""
        function_release = ""
    else:
        automation_id = crypto_random_string(10)
        automation_revision_id = crypto_random_string(10)
        automation_run_id = crypto_random_string(11)
        function_id = crypto_random_string(10)
        function_release = crypto_random_string(10)

    automation_context = AutomationContext.initialize(
        AutomationRunData(
            project_id="3354d7be31",
            model_id="724617c963",
            branch_name="example.ifc",
            version_id="2a514e194f",
            speckle_server_url="https://latest.speckle.systems",
            automation_id=automation_id,
            automation_revision_id=automation_revision_id,
            automation_run_id=automation_run_id,
            function_id=function_id,
            function_release=function_release,
        ),
        speckle_token,
    )

    register_new_automation(
        automation_context.speckle_client,
        automation_context.automation_run_data.project_id,
        automation_context.automation_run_data.model_id,
        automation_context.automation_run_data.automation_id,
        "Jupyter local test",
        automation_context.automation_run_data.automation_revision_id,
    )
    print("local test registered")

local test registered


# Function inputs

The `FunctionInputs` class defines the schema for the values, this function requires to run.
These values will be provided by the users of the function.

Automate uses the Json Schema, generated from the class, to validate user provided values.

The schema block is tagget with the `function_input` tag, do not remove that!


In [61]:
from speckle_automate import AutomateBase
from pydantic import Field
import json


class FunctionInputs(AutomateBase):
    """These are function author defined values.

    Automate will make sure to supply them matching the types specified here.
    Please use the pydantic model schema to define your inputs:
    https://docs.pydantic.dev/latest/usage/models/
    """

    forbidden_speckle_type: str = Field(
        title="Forbidden speckle type",
        description=(
            "If a object has the following speckle_type,"
            " it will be marked with an error."
        ),
    )


print(json.dumps(FunctionInputs.model_json_schema()))

{"description": "These are function author defined values.\n\nAutomate will make sure to supply them matching the types specified here.\nPlease use the pydantic model schema to define your inputs:\nhttps://docs.pydantic.dev/latest/usage/models/", "properties": {"forbiddenSpeckleType": {"description": "If a object has the following speckle_type, it will be marked with an error.", "title": "Forbidden speckle type", "type": "string"}}, "required": ["forbiddenSpeckleType"], "title": "FunctionInputs", "type": "object"}


# Function logic

in this block we're defining the actual business logic of the function.

By all means modify this block but do keep the function signature compatible with the executing main function.


In [62]:
from specklepy.objects import Base
from typing import Iterable

from speckle_automate import AutomationContext


def flatten_base(base: Base) -> Iterable[Base]:
    """Take a base and flatten it to an iterable of bases."""
    if hasattr(base, "elements"):
        for element in base["elements"]:
            yield from flatten_base(element)
    yield base


def automate_function(
    automate_context: AutomationContext,
    function_inputs: FunctionInputs,
):
    version_root_object = automate_context.receive_version()

    count = 0
    for b in flatten_base(version_root_object):
        if b.speckle_type == function_inputs.forbidden_speckle_type:
            if not b.id:
                raise ValueError("Cannot operate on objects without their id's.")
            automate_context.add_object_error(
                b.id,
                "This project should not contain the type: " f"{b.speckle_type}",
            )
            count += 1

    if count > 0:
        # this is how a run is marked with a failure cause
        automate_context.mark_run_failed(
            "Automation failed: "
            f"Found {count} object that have one of the forbidden speckle types: "
            f"{function_inputs.forbidden_speckle_type}"
        )

    else:
        automate_context.mark_run_success("No forbidden types found.")

# Function execution

This last bit executes the actual function using the automation context.
Do not change stuff unless you know what you are doing.


In [63]:
import os
from speckle_automate.runner import run_function

if execute_function:
    automation_context = run_function(
        AutomationContext.initialize(automation_run_data, os.environ[token_env_var]),
        automate_function,
        FunctionInputs.model_validate_json(function_inputs),
    )

    # WARNING: Make sure the last print statement is this!
    # automate relies on this convention, to process the automation results
    print(automation_context._automation_result.model_dump_json())

It took 2.040990 seconds to receive  the speckle version 2a514e194f
Automation run SUCCEEDED after 2.044367 seconds.
No forbidden types found.
Run completed with status: AutomationStatus.SUCCEEDED, message: No forbidden types found.
